In [1]:
# -*- coding: utf-8 -*-
# @Time : 2020/11/4
# @Author :
# @File : textcnn.py
# @Software: PyCharm

import  tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, GlobalAveragePooling1D, Dense, Concatenate, GlobalMaxPooling1D
from tensorflow.keras import Model

class TextCNN(Model):

    def __init__(self,
                 maxlen,
                 max_features,
                 embedding_dims,
                 class_num,
                 kernel_sizes=[1,2,3],
                 kernel_regularizer=None,
                 last_activation='softmax'
                 ):
        '''
        :param maxlen: 文本最大长度
        :param max_features: 词典大小
        :param embedding_dims: embedding维度大小
        :param kernel_sizes: 滑动卷积窗口大小的list, eg: [1,2,3]
        :param kernel_regularizer: eg: tf.keras.regularizers.l2(0.001)
        :param class_num:
        :param last_activation:
        '''
        super(TextCNN, self).__init__()
        self.maxlen = maxlen
        self.kernel_sizes = kernel_sizes
        self.class_num = class_num
        self.embedding = Embedding(input_dim=max_features, output_dim=embedding_dims, input_length=maxlen)
        self.conv1s = []
        self.avgpools = []
        for kernel_size in kernel_sizes:
            self.conv1s.append(Conv1D(filters=128, kernel_size=kernel_size, activation='relu', kernel_regularizer=kernel_regularizer))
            self.avgpools.append(GlobalMaxPooling1D())
        self.classifier = Dense(class_num, activation=last_activation, )

    def call(self, inputs, training=None, mask=None):
        if len(inputs.get_shape()) != 2:
            raise ValueError('The rank of inputs of TextCNN must be 2, but now is %d' % len(inputs.get_shape()))
        if inputs.get_shape()[1] != self.maxlen:
            raise ValueError('The maxlen of inputs of TextCNN must be %d, but now is %d' % (self.maxlen, inputs.get_shape()[1]))

        emb = self.embedding(inputs)
        conv1s = []
        for i in range(len(self.kernel_sizes)):
            c = self.conv1s[i](emb) # (batch_size, maxlen-kernel_size+1, filters)
            c = self.avgpools[i](c) # # (batch_size, filters)
            conv1s.append(c)
        x = Concatenate()(conv1s) # (batch_size, len(self.kernel_sizes)*filters)
        output = self.classifier(x)
        return output

    def build_graph(self, input_shape):
        '''自定义函数，在调用model.summary()之前调用
        '''
        input_shape_nobatch = input_shape[1:]
        self.build(input_shape)
        inputs = tf.keras.Input(shape=input_shape_nobatch)
        if not hasattr(self, 'call'):
            raise AttributeError("User should define 'call' method in sub-class model!")
        _ = self.call(inputs)


In [2]:
# ===================== set random  ===========================
import numpy as np
import tensorflow as tf
import random as rn
np.random.seed(0)
rn.seed(0)
tf.random.set_seed(0)
# =============================================================

import os
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
def checkout_dir(dir_path, do_delete=False):
    import shutil
    if do_delete and os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    if not os.path.exists(dir_path):
        print(dir_path, 'make dir ok')
        os.makedirs(dir_path)


class ModelHepler:
    def __init__(self, class_num, maxlen, max_features, embedding_dims, epochs, batch_size, last_activation = 'softmax'):
        self.class_num = class_num
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.epochs = epochs
        self.batch_size = batch_size
        self.callback_list = []
        self.last_activation = last_activation
        print('Bulid Model...')
        self.create_model()

    def create_model(self):
        model = TextCNN(maxlen=self.maxlen,
                         max_features=self.max_features,
                         embedding_dims=self.embedding_dims,
                         class_num=self.class_num,
                         kernel_sizes=[2,3,5],
                         kernel_regularizer=None,
                         last_activation=self.last_activation)
        if self.last_activation == 'softmax':
          model.compile(
              optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'],
          )
        else:
          model.compile(
              optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
          )

        model.build_graph(input_shape=(None, maxlen))
        model.summary()
        self.model =  model

    def get_callback(self, use_early_stop=True, tensorboard_log_dir='logs\\TextCNN-epoch-5', checkpoint_path="save_model_dir\\cp-moel.ckpt"):
        callback_list = []
        if use_early_stop:
            # EarlyStopping
            early_stopping = EarlyStopping(monitor='val_accuracy', patience=7, mode='max')
            callback_list.append(early_stopping)
        if checkpoint_path is not None:
            # save model
            checkpoint_dir = os.path.dirname(checkpoint_path)
            checkout_dir(checkpoint_dir, do_delete=True)
            # 创建一个保存模型权重的回调
            cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                             monitor='val_accuracy',
                                             mode='max',
                                             save_best_only=True,
                                             save_weights_only=True,
                                             verbose=1,
                                             period=2,
                                             )
            callback_list.append(cp_callback)
        if tensorboard_log_dir is not None:
            # tensorboard --logdir logs/TextCNN-epoch-5
            checkout_dir(tensorboard_log_dir, do_delete=True)
            tensorboard_callback = TensorBoard(log_dir=tensorboard_log_dir, histogram_freq=1)
            callback_list.append(tensorboard_callback)
        self.callback_list = callback_list

    def fit(self, x_train, y_train, x_val, y_val):
        print('Train...')
        self.model.fit(x_train, y_train,
                  batch_size=self.batch_size,
                  epochs=self.epochs,
                  verbose=2,
                  callbacks=self.callback_list,
                  validation_data=(x_val, y_val))

    def load_model(self, checkpoint_path):
        checkpoint_dir = os.path.dirname((checkpoint_path))
        latest = tf.train.latest_checkpoint(checkpoint_dir)
        print('restore model name is : ', latest)
        # 创建一个新的模型实例
        # model = self.create_model()
        # 加载以前保存的权重
        self.model.load_weights(latest)

2.0.0


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
# ================  params =========================
class_num = 6
maxlen = 300
embedding_dims = 200
epochs = 100
batch_size = 64
max_features = 50000

MODEL_NAME = 'company_manager_relation_TextCNN'
use_early_stop=True

Loading data...
Pad sequences (samples x time)...
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Bulid Model...
Model: "text_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 400, 200)          1000000   
_________________________________________________________________
conv1d (Conv1D)              (None, 399, 128)          51328     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 398, 128)          76928     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 396, 128)          128128    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
____

NotFoundError: Failed to create a directory: ./logs/company_manager_relation_TextCNN\train\plugins\profile\2020-11-04_12-10-30; No such file or directory

In [ ]:
print('Loading data...')
tensorboard_log_dir = os.path.join(path, 'logs/{}'.format(MODEL_NAME))
# checkpoint_path = "save_model_dir\\{}\\cp-{epoch:04d}.ckpt".format(MODEL_NAME, '')
checkpoint_path = os.path.join(path, 'save_model_dir/'+MODEL_NAME+'/cp-{epoch:04d}.ckpt')
#  ====================================================================

data_file = os.path.join(path, 'data/comp_manager_data_grouped.csv')
import pandas as pd
data = pd.read_csv(data_file, encoding='utf-8')

x_train = data[data['group']=='train']['sentence'].values
train_token_ids = [[tokenizer.convert_tokens_to_ids(x) for x in tokenizer.tokenize(text)] for text in x_train]
x_train = pad_sequences(train_token_ids, maxlen=maxlen, padding='post')

y_train = data[data['group']=='train'][['output_analyst','output_chief_investment_officer',	
                                            'output_managing_director',	'output_other',	
                                            'output_portfolio_manager',	'output_vice_president']].values

x_test = data[data['group']=='test']['sentence'].values
test_token_ids = [[tokenizer.convert_tokens_to_ids(x) for x in tokenizer.tokenize(text)] for text in x_test]
x_test = pad_sequences(test_token_ids, maxlen=maxlen, padding='post')

y_test = data[data['group']=='test'][['output_analyst','output_chief_investment_officer',	
                                            'output_managing_director',	'output_other',	
                                            'output_portfolio_manager',	'output_vice_president']].values

x_val = data[data['group']=='dev']['sentence'].values
val_token_ids = [[tokenizer.convert_tokens_to_ids(x) for x in tokenizer.tokenize(text)] for text in x_val]
x_val = pad_sequences(val_token_ids, maxlen=maxlen, padding='post')
y_val = data[data['group']=='dev'][['output_analyst','output_chief_investment_officer',	
                                            'output_managing_director',	'output_other',	
                                            'output_portfolio_manager',	'output_vice_president']].values

print('Pad sequences (samples x time)...')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('x_val shape:', x_val.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print('y_val shape:', y_val.shape)

In [ ]:
model_hepler = ModelHepler(class_num=class_num,
                           maxlen=maxlen,
                           max_features=max_features,
                           embedding_dims=embedding_dims,
                           epochs=epochs,
                           batch_size=batch_size,
                           last_activation='sigmoid')
model_hepler.get_callback(use_early_stop=use_early_stop, tensorboard_log_dir=tensorboard_log_dir, checkpoint_path=checkpoint_path)
model_hepler.fit(x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val)
print('Test...')
result = model_hepler.model.predict(x_test)
test_score = model_hepler.model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print("test loss:", test_score[0], "test accuracy", test_score[1])

In [ ]:
model_hepler = ModelHepler(class_num=class_num,
                           maxlen=maxlen,
                           max_features=max_features,
                           embedding_dims=embedding_dims,
                           epochs=epochs,
                           batch_size=batch_size,
                           last_activation='sigmoid')
model_hepler.load_model(checkpoint_path=checkpoint_path)
# 重新评估模型
loss, acc = model_hepler.model.evaluate(x_test, y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))